# CS229: Problem Set 1
## Problem 3: Gaussian Discriminant Analysis


**C. Combier**

This iPython Notebook provides solutions to Stanford's CS229 (Machine Learning, Fall 2017) graduate course problem set 1, taught by Andrew Ng.

The problem set can be found here: [./ps1.pdf](ps1.pdf)

I chose to write the solutions to the coding questions in Python, whereas the Stanford class is taught with Matlab/Octave.

## Notation

- $x^i$ is the $i^{th}$ feature vector
- $y^i$ is the expected outcome for the $i^{th}$ training example
- $m$ is the number of training examples
- $n$ is the number of features

### Question 3.a)

The gist of the solution is simply to apply Bayes rule, and simplify the exponential terms in the denominator which gives us the sigmoid function. The calculations are somewhat heavy:

$$
\begin{align*}
p(y=1 \mid x) & = \frac{p(x \mid y=1)p(y=1)}{p(x)} \\
              & = \frac{p(x \mid y=1)p(y=1)}{p(x \mid y=1)p(y=1)+ p(x \mid y=-1)p(y=-1)} \\
              & = \frac{\frac{1}{(2\pi)^{\frac{n}{2}} \lvert \Sigma \rvert^{\frac{1}{2}}} \exp \left(-\frac{1}{2} \left(x-\mu_{1} \right)^T\Sigma^{-1} \left(x-\mu_{1} \right) \right) \phi  }{ \frac{1}{(2\pi)^{\frac{n}{2}} \lvert \Sigma \rvert^{\frac{1}{2}}} \exp \left(-\frac{1}{2} \left(x-\mu_{1} \right)^T\Sigma^{-1} \left(x-\mu_{1} \right) \right) \phi + \frac{1}{(2\pi)^{\frac{n}{2}} \lvert \Sigma \rvert^{\frac{1}{2}}} \exp \left(-\frac{1}{2} \left(x-\mu_{-1} \right)^T\Sigma^{-1} \left(x-\mu_{-1} \right) \right)\left(1-\phi \right)} \\
              & = \frac{\phi \exp \left(-\frac{1}{2} \left(x-\mu_{1} \right)^T\Sigma^{-1} \left(x-\mu_{1} \right) \right) }{\phi \exp \left(-\frac{1}{2} \left(x-\mu_{1} \right)^T\Sigma^{-1} \left(x-\mu_{1} \right) \right)  +  \left(1-\phi \right) \exp \left(-\frac{1}{2} \left(x-\mu_{-1} \right)^T\Sigma^{-1} \left(x-\mu_{-1} \right) \right)} \\
              & = \frac{1}{1+ \exp \left(\log\left(\frac{\left(1-\phi \right)}{\phi}\right) -\frac{1}{2} \left(x-\mu_{-1} \right)^T\Sigma^{-1} \left(x-\mu_{-1} \right) + \frac{1}{2} \left(x-\mu_{1} \right)^T\Sigma^{-1} \left(x-\mu_{1} \right) \right)} \\
              & = \frac{1}{1+\exp \left(\log \left(\frac{1-\phi}{\phi}\right) -\frac{1}{2} \left(x^T \Sigma^{-1}x -2x^T \Sigma^{-1}\mu_{-1}+ \mu_{-1}^T \Sigma^{-1} \mu_{-1}\right) + \frac{1}{2} \left(x^T \Sigma^{-1}x -2x^T \Sigma^{-1}\mu_{1}+ \mu_{1}^T \Sigma^{-1} \mu_{1} \right)\right)} \\
              & = \frac{1}{1+\exp \left(\log \left(\frac{1-\phi}{\phi}\right) + x^T \Sigma^{-1} \mu_{-1} - x^T \Sigma^{-1} \mu_1 - \frac{1}{2} \mu_{-1}^T \Sigma^{-1} \mu_{-1} + \frac{1}{2} \mu_1^T\Sigma^{-1}\mu_1 \right)} \\
              & = \frac{1}{1+ \exp\left(\log\left(\frac{1-\phi}{\phi}\right) + x^T \Sigma^{-1} \left(\mu_{-1}-\mu_1 \right) - \frac{1}{2}\mu_{-1}^T\Sigma^{-1}\mu_{-1} + \mu_1^T \Sigma^{-1} \mu_1 \right)} \\
              \\
\end{align*}
$$

With:
- $\theta_0 = \frac{1}{2}\left(\mu_{-1}^T \Sigma^{-1} \mu_{-1}- \mu_1^T \Sigma^{-1}\mu_1 \right)-\log\frac{1-\phi}{\phi} $
- $\theta = \Sigma^{-1}\left(\mu_{1}-\mu_{-1} \right)$

we have:

$$
p(y=1 \mid x) = \frac{1}{1+\exp \left(-y(\theta^Tx + \theta_0) \right)}
$$

### Questions 3.b) and 3.c)
Question 3.b) is the special case where $n=1$. Let us prove the general case directly, as required in 3.c):

$$
\begin{align*}
\ell \left(\phi, \mu_{-1}, \mu_1, \Sigma \right)  & = \log \prod_{i=1}^m p(x^{i}\mid y^i; \phi, \mu_{-1}, \mu_1, \Sigma)p(y^{i};\phi) \\
                                                  & = \sum_{i=1}^m \log p(x^{i}\mid y^{i}; \phi, \mu_{-1}, \mu_1, \Sigma) + \sum_{i=1}^m \log p(y^{i};\phi) \\
                                                  & = \sum_{i=1}^m \left[\log \frac{1}{\left(2 \pi \right)^{\frac{n}{2}} \lvert \Sigma \rvert^{\frac{1}{2}}} - \frac{1}{2} \left(x^{i} - \mu_{y^{i}}  \right)^T \Sigma^{-1} \left(x^{i} - \mu_{y^{i}}  \right) + \log \phi^{y^{i}} + \log \left(1- \phi \right)^{\left(1-y^{i} \right)} \right] \\
                                                  & \simeq \sum_{i=1}^m \left[- \frac{1}{2} \log \lvert \Sigma \rvert - \frac{1}{2} \left(x^{i} - \mu_{y^{i}}  \right)^T \Sigma^{-1} \left(x^{i} - \mu_{y^{i}}  \right) + y^{i} \log \phi + \left(1-y^{i} \right) \log \left(1- \phi \right) \right] \\
\end{align*}
$$

Now we calculate the maximum likelihood be calculating the gradient of the log-likelihood with respect to the parameters and setting it to $0$:

$$
\begin{align*}
\frac{\partial \ell}{\partial \phi} &= \sum_{i=1}^{m}( \frac{y^i}{\phi} - \frac{1-y^i}{1-\phi}) \\
&= \sum_{i=1}^{m}\frac{1(y^i = 1)}{\phi} + \frac{m-\sum_{i=1}^{m}1(y^i = 1)}{1-\phi}
\end{align*}
$$

Therefore, $\phi = \frac{1}{m} \sum_{i=1}^m 1(y^i =1 )$, i.e. the percentage of the training examples such that $y^i = 1$

Now for $\mu_{-1}:$
$$
\begin{align*}
\nabla_{\mu_{-1}} \ell  & = - \frac{1}{2} \sum_{i : y^{i}=-1} \nabla_{\mu_{-1}} \left[ -2 \mu_{-1}^T \Sigma^{-1} x^{(i)} + \mu_{-1}^T \Sigma^{-1} \mu_{-1}  \right] \\
                                                                    & = - \frac{1}{2} \sum_{i : y^{i}=-1} \left[-2 \Sigma^{-1}x^{(i)} + 2  \Sigma^{-1} \mu_{-1} \right]
\end{align*}
$$

Again, we set the gradient to $0$:

$$
\begin{align*}
                                                                  \sum_{i:y^i=-1} \left[\Sigma^{-1}x^{i}-\Sigma^{-1} \mu_{-1} \right] &= 0 \\
                                                                  \sum_{i=1}^m 1 \left\{y^{i}=-1\right\} \Sigma^{-1} x^{(i)} - \sum_{i=1}^m 1 \left\{y^{i}=-1 \right\} \Sigma^{-1} \mu_{-1} &=0 \\
\end{align*}
$$
This yields:
$$
\Sigma^{-1} \mu_{-1} \sum_{i=1}^m 1 \left\{y^{i}=-1 \right\} = \Sigma^{-1} \sum_{i=1}^m 1 \left\{y^{(i)}=-1\right\}  x^{i}
$$
Allowing us to finally write:
$$\mu_{-1} = \frac{\sum_{i=1}^m 1 \left\{y^{i}=-1\right\}  x^{i}}{\sum_{i=1}^m 1 \left\{y^{(i)}=-1 \right\}}$$

The calculations are similar for $\mu_1$, and we obtain:
$$\mu_{1} = \frac{\sum_{i=1}^m 1 \left\{y^{i}=1\right\}  x^{i}}{\sum_{i=1}^m 1 \left\{y^{i}=1 \right\}}$$


The last step is to calculate the gradient with respect to $\Sigma$. To simplify calculations, let us calculate the gradient for $S = \frac{1}{\Sigma}$.

$$
\begin{align*}
\nabla_{S} \ell  & = - \frac{1}{2}\sum_{i=1}^m \nabla_{\Sigma} \left[-\log \lvert S \rvert + \left(x^{i}- \mu_{y^{i}} \right)^T S \left(x^{i}- \mu_{y^{i}} \right) \right] \\
                                                                  & = - \frac{1}{2}\sum_{i=1}^m \left[-S^{-1} + \left(x^{i}- \mu_{y^{i}} \right)\left(x^{i}- \mu_{y^{i}} \right)^T \right] \\
                                                                  & = \sum_{i=1}^m \frac{1}{2} \Sigma - \frac{1}{2} \sum_{i=1}^m \left(x^{i}- \mu_{y^{i}} \right)\left(x^{i}- \mu_{y^{i}} \right)^T\\
\end{align*}
$$
Again, we set the gradient to $0$, allowing us to write:
$$
\frac{1}{2} m \Sigma = \frac{1}{2} \sum_{i=1}^m \left(x^{i}- \mu_{y^{i}} \right)\left(x^{i}- \mu_{y^{i}} \right)^T \\
$$
Finally, we obtain the maximum likelihood estimate for $\Sigma$:
$$
\Sigma = \frac{1}{m}\sum_{i=1}^m \left(x^{i}- \mu_{y^{i}} \right)\left(x^{i}- \mu_{y^{i}} \right)^T
$$
